In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

In [ ]:
# import dataframes
path = r'...'
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))
orders_products_flagged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_flagged_4.8.pkl'))







# Wrangling

In [3]:
customers.shape

(206209, 10)

In [4]:
# check column (names)
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
# renaming columns (lowercase, underscore, spelling mistake)
customers.rename(columns={'user_id' : 'customer_id'}, inplace = True)
customers.rename(columns={'First Name' : 'first_name'}, inplace = True)
customers.rename(columns={'Surnam' : 'last_name'}, inplace = True)
customers.rename(columns={'Gender' : 'gender'}, inplace = True)
customers.rename(columns={'STATE' : 'state'}, inplace = True)
customers.rename(columns={'Age' : 'age'}, inplace = True)
customers.rename(columns={'fam_status' : 'marital_status'}, inplace = True)

In [6]:
customers.head()

,customer_id,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# Data Consistency

In [7]:
# check for NaNs
customers.isnull().sum()

customer_id           0
first_name        11259
last_name             0
gender                0
state                 0
age                   0
date_joined           0
n_dependants          0
marital_status        0
income                0
dtype: int64

In [8]:
# view missing data
customers_nan = customers[customers['first_name'].isnull() == True]

In [9]:
customers_nan

,customer_id,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819
...,...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3/31/2020,3,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,4/1/2020,2,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,4/1/2020,1,married,45275
206162,187532,NaN,Floyd,Female,California,39,4/1/2020,0,single,56325


In [10]:
# check for duplicates
customers[customers.duplicated()]

,customer_id,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income


###### there are NO duplicates

In [11]:
# check for mixed data
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

first_name


###### there is mixed data in 'first_name' - maybe because of the NaNs

In [12]:
# get data type of 'first_name'
customers['first_name'].dtype

dtype('O')

In [13]:
customers['first_name'] = customers['first_name'].astype('str')

In [14]:
# check data type of 'first_name'
customers['first_name'].dtype

dtype('O')

In [15]:
# check for mixed data
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

In [16]:
# check datatypes
customers.dtypes

customer_id        int64
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
n_dependants       int64
marital_status    object
income             int64
dtype: object

In [17]:
# further checks
customers.describe()

,customer_id,age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [19]:
customers.shape

(206209, 10)

# Combine Data

In [21]:
orders_products_flagged.head()

,order_id,customer_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,price_range,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_order,spending_flag,days_since_prior_order_median,order_frequency_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,5.813559,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,False,196,1,1,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,5.813559,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,False,196,1,1,...,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,5.813559,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,False,196,1,1,...,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,5.813559,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,False,196,1,1,...,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,5.813559,Low spender,20.5,Non-frequent customer


In [22]:
orders_products_flagged['customer_id'].dtype

dtype('int64')

In [23]:
customers.head()

,customer_id,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [24]:
customers['customer_id'].dtype

dtype('int64')

In [25]:
orders_products_customers = orders_products_flagged.merge(customers, on = 'customer_id')

In [26]:
orders_products_customers

,order_id,customer_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,order_frequency_flag,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,False,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,False,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,False,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,False,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,156685,106143,26,4,23,5.0,False,19675,1,1,...,Frequent customer,Gerald,Yates,Male,Hawaii,25,5/26/2017,0,single,53755
32404855,484769,66343,1,6,11,NaN,True,47210,1,0,...,Non-frequent customer,Jacqueline,Arroyo,Female,Tennessee,22,9/12/2017,3,married,46151
32404856,1561557,66343,2,1,11,30.0,False,47210,1,1,...,Non-frequent customer,Jacqueline,Arroyo,Female,Tennessee,22,9/12/2017,3,married,46151
32404857,276317,66343,3,6,15,19.0,False,47210,1,1,...,Non-frequent customer,Jacqueline,Arroyo,Female,Tennessee,22,9/12/2017,3,married,46151


In [27]:
# export to pkl
orders_products_customers.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers.pkl'))